In [2]:
import pandas as pd

# Import the four dataframes
df_artists = pd.read_csv('/Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/df_artists.csv')
df_songs = pd.read_csv('/Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/df_songs.csv')
df_albums = pd.read_csv('/Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/df_albums.csv')
df_artists_top_10_songs_only = pd.read_csv('/Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/df_artists_top_10_songs_only.csv')

# Display basic info about each dataframe
print(f"df_artists shape: {df_artists.shape}")
print(f"df_songs shape: {df_songs.shape}")
print(f"df_albums shape: {df_albums.shape}")
print(f"df_artists_top_10_songs_only shape: {df_artists_top_10_songs_only.shape}")


df_artists shape: (14226, 54)
df_songs shape: (5099, 16)
df_albums shape: (4569, 16)
df_artists_top_10_songs_only shape: (2420, 54)


In [12]:
import psycopg2

DB_PARAMS = {
    'dbname': 'postgres',  # Connect to default postgres database
    'user': 'musicbrainz',
    'password': 'musicbrainz',
    'host': 'localhost',
    'port': 5432
}

try:
    conn = psycopg2.connect(**DB_PARAMS)
    conn.autocommit = True
    
    with conn.cursor() as cur:
        # List all databases
        cur.execute("""
            SELECT datname 
            FROM pg_database 
            WHERE datistemplate = false
            ORDER BY datname
        """)
        databases = cur.fetchall()
        
        print("Available databases:")
        for db in databases:
            print(f"  {db[0]}")
    
    conn.close()
except Exception as e:
    print(f"Error: {e}")


Available databases:
  musicbrainz
  musicbrainz_db
  postgres


In [16]:
import pandas as pd
import psycopg2
from tqdm import tqdm

# Database connection parameters
DB_PARAMS = {
    'dbname': 'musicbrainz_db',
    'user': 'musicbrainz',
    'password': 'musicbrainz',
    'host': 'localhost',
    'port': 5432
}

print("Step 1: Loading all dataframes...")
df_artists = pd.read_csv('/Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/df_artists.csv')
df_songs = pd.read_csv('/Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/df_songs.csv')
df_albums = pd.read_csv('/Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/df_albums.csv')
df_artists_top_10_songs_only = pd.read_csv('/Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/df_artists_top_10_songs_only.csv')

print(f"  df_artists: {df_artists.shape}")
print(f"  df_songs: {df_songs.shape}")
print(f"  df_albums: {df_albums.shape}")
print(f"  df_artists_top_10_songs_only: {df_artists_top_10_songs_only.shape}")

# Step 2: Get all unique artist names
print("\nStep 2: Collecting unique artist names...")
all_artist_names = set()

if 'performer_normalized' in df_artists.columns:
    all_artist_names.update(df_artists['performer_normalized'].dropna().unique())

if 'performer_normalized' in df_songs.columns:
    all_artist_names.update(df_songs['performer_normalized'].dropna().unique())

if 'performer_normalized' in df_albums.columns:
    all_artist_names.update(df_albums['performer_normalized'].dropna().unique())

if 'performer_normalized' in df_artists_top_10_songs_only.columns:
    all_artist_names.update(df_artists_top_10_songs_only['performer_normalized'].dropna().unique())

all_artist_names = list(all_artist_names)
print(f"  Found {len(all_artist_names)} unique artist names to look up")

# Step 3: Query MusicBrainz database for artist IDs in batches
print("\nStep 3: Querying MusicBrainz database for artist IDs (batch mode)...")

artist_name_to_id = {}
batch_size = 1000  # Process 1000 names at a time

conn = psycopg2.connect(**DB_PARAMS)
conn.autocommit = True

num_batches = (len(all_artist_names) + batch_size - 1) // batch_size

with conn.cursor() as cur:
    for i in tqdm(range(num_batches), desc="Processing batches"):
        batch_names = all_artist_names[i * batch_size:(i + 1) * batch_size]
        
        # Single query for the entire batch using unnest and join
        query = """
            SELECT DISTINCT ON (LOWER(input_name)) 
                input_name, 
                artist.id, 
                artist.name
            FROM unnest(%s::text[]) AS input_name
            LEFT JOIN artist ON LOWER(artist.name) = LOWER(input_name)
            WHERE artist.id IS NOT NULL
        """
        
        cur.execute(query, (batch_names,))
        results = cur.fetchall()
        
        # Map results
        for input_name, artist_id, matched_name in results:
            artist_name_to_id[input_name] = str(artist_id)

conn.close()

# Find which artists weren't matched
not_found = [name for name in all_artist_names if name not in artist_name_to_id]

print(f"  Successfully matched: {len(artist_name_to_id):,} artists")
print(f"  Not found in MusicBrainz: {len(not_found):,} artists")

if not_found:
    print(f"\nSample of artists not found (first 10):")
    for name in not_found[:10]:
        print(f"    - {name}")

# Step 4: Add musicbrainz_artist_id column to all dataframes
print("\nStep 4: Adding musicbrainz_artist_id column to all dataframes...")

def add_mb_id_column(df, name):
    if 'performer_normalized' in df.columns:
        df['musicbrainz_artist_id'] = df['performer_normalized'].map(artist_name_to_id)
        matched = df['musicbrainz_artist_id'].notna().sum()
        total = len(df)
        print(f"  {name}: {matched:,}/{total:,} rows matched ({matched/total*100:.1f}%)")
    else:
        print(f"  {name}: No 'performer_normalized' column found")
    return df

df_artists = add_mb_id_column(df_artists, 'df_artists')
df_songs = add_mb_id_column(df_songs, 'df_songs')
df_albums = add_mb_id_column(df_albums, 'df_albums')
df_artists_top_10_songs_only = add_mb_id_column(df_artists_top_10_songs_only, 'df_artists_top_10_songs_only')

# Step 5: Save updated dataframes
print("\nStep 5: Saving updated dataframes...")
df_artists.to_csv('/Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/df_artists.csv', index=False)
print("  ✓ Saved df_artists.csv")

df_songs.to_csv('/Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/df_songs.csv', index=False)
print("  ✓ Saved df_songs.csv")

df_albums.to_csv('/Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/df_albums.csv', index=False)
print("  ✓ Saved df_albums.csv")

df_artists_top_10_songs_only.to_csv('/Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/df_artists_top_10_songs_only.csv', index=False)
print("  ✓ Saved df_artists_top_10_songs_only.csv")

print("\n" + "="*80)
print("COMPLETE: All dataframes now have 'musicbrainz_artist_id' column")
print("="*80)


Step 1: Loading all dataframes...
  df_artists: (14226, 54)
  df_songs: (5099, 16)
  df_albums: (4569, 16)
  df_artists_top_10_songs_only: (2420, 54)

Step 2: Collecting unique artist names...
  Found 14226 unique artist names to look up

Step 3: Querying MusicBrainz database for artist IDs (batch mode)...


Processing batches: 100%|██████████| 15/15 [00:27<00:00,  1.81s/it]

  Successfully matched: 11,564 artists
  Not found in MusicBrainz: 2,662 artists

Sample of artists not found (first 10):
    - tsol
    - the dropkick murphys
    - auli'i cravalho
    - beele
    - k.w.s.
    - danyel gerard
    - persuasions
    - '68
    - 21 savage & tyler, the creator
    - steve stevens atomic playboys

Step 4: Adding musicbrainz_artist_id column to all dataframes...
  df_artists: 11,564/14,226 rows matched (81.3%)
  df_songs: 4,500/5,099 rows matched (88.3%)
  df_albums: 4,181/4,569 rows matched (91.5%)
  df_artists_top_10_songs_only: 2,041/2,420 rows matched (84.3%)

Step 5: Saving updated dataframes...
  ✓ Saved df_artists.csv
  ✓ Saved df_songs.csv
  ✓ Saved df_albums.csv
  ✓ Saved df_artists_top_10_songs_only.csv

COMPLETE: All dataframes now have 'musicbrainz_artist_id' column


In [25]:
import pandas as pd
import psycopg2
from psycopg2 import sql
from tqdm import tqdm
import os
from pathlib import Path

# Database connection parameters
DB_PARAMS = {
    'dbname': 'musicbrainz_db',
    'user': 'musicbrainz',
    'password': 'musicbrainz',
    'host': 'localhost',
    'port': 5432
}

# ============================================================================
# LOAD BILLBOARD ARTIST IDs
# ============================================================================

print("Loading Billboard artist IDs...")
df_artists_top_10 = pd.read_csv('/Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/df_artists_top_10_songs_only.csv')

# Get artist IDs and filter out NaN values, convert to integers
artist_ids = df_artists_top_10['musicbrainz_artist_id'].dropna().astype(int).unique().tolist()

print(f"Total artists in df_artists_top_10_songs_only: {len(df_artists_top_10)}")
print(f"Artists with MusicBrainz IDs: {len(artist_ids)}")
print(f"Artists without MusicBrainz IDs: {df_artists_top_10['musicbrainz_artist_id'].isna().sum()}")

# ============================================================================
# PHASE 1: Extract Master Edge List
# ============================================================================

print("\n" + "="*80)
print("PHASE 1: Extracting Master Edge List from MusicBrainz")
print("="*80)

def extract_master_edge_list(artist_ids, batch_size=500):
    """
    Extract all collaboration edges for the given artist IDs.
    Uses batching to avoid memory issues with large IN clauses.
    """
    
    # SQL query template - Uses INTEGER type, not UUID
    sql_query = """
    WITH EarliestRG AS (
        -- Anchor every album/single to its original release year
        SELECT r.release_group, MIN(rc.date_year) as first_year
        FROM release r
        JOIN release_country rc ON r.id = rc.release
        WHERE rc.date_year IS NOT NULL
        GROUP BY r.release_group
    ),
    CollaborationEdges AS (
        -- Find every unique pair of artists on the same credit
        SELECT 
            acn1.artist AS artist_a,
            acn2.artist AS artist_b,
            rg.id AS release_group_id,
            erg.first_year
        FROM artist_credit_name acn1
        JOIN artist_credit_name acn2 ON acn1.artist_credit = acn2.artist_credit
        JOIN release_group rg ON rg.artist_credit = acn1.artist_credit
        JOIN EarliestRG erg ON rg.id = erg.release_group
        WHERE acn1.artist = ANY(%s::integer[])
          AND acn1.artist < acn2.artist
    )
    SELECT * FROM CollaborationEdges;
    """
    
    all_edges = []
    
    # Process in batches
    num_batches = (len(artist_ids) + batch_size - 1) // batch_size
    
    conn = psycopg2.connect(**DB_PARAMS)
    conn.autocommit = True
    
    with conn.cursor() as cur:
        for i in tqdm(range(num_batches), desc="Extracting batches"):
            batch_ids = artist_ids[i*batch_size:(i+1)*batch_size]
            
            # Execute query for this batch
            cur.execute(sql_query, (batch_ids,))
            
            # Fetch results
            batch_results = cur.fetchall()
            all_edges.extend(batch_results)
            
            print(f"  Batch {i+1}/{num_batches}: Found {len(batch_results):,} edges")
    
    conn.close()
    
    # Convert to DataFrame
    df_edges = pd.DataFrame(all_edges, columns=['artist_a', 'artist_b', 'release_group_id', 'first_year'])
    
    return df_edges

# Extract the master edge list
print("\nExecuting SQL query (this may take several minutes)...")
master_edge_list = extract_master_edge_list(artist_ids)

print(f"\nTotal edges extracted: {len(master_edge_list):,}")
print(f"Year range: {master_edge_list['first_year'].min()} - {master_edge_list['first_year'].max()}")
print(f"Unique artist pairs: {master_edge_list[['artist_a', 'artist_b']].drop_duplicates().shape[0]:,}")

# Save to parquet
output_path = '/Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/master_edge_list.parquet'
master_edge_list.to_parquet(output_path, index=False)
print(f"\nMaster edge list saved to: {output_path}")

# Also save as CSV for easy inspection
csv_path = '/Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/master_edge_list.csv'
master_edge_list.to_csv(csv_path, index=False)
print(f"Also saved as CSV: {csv_path}")

# ============================================================================
# PHASE 2: Generate Temporal Network Snapshots
# ============================================================================

print("\n" + "="*80)
print("PHASE 2: Generating Temporal Network Snapshots")
print("="*80)

# Create output directory structure
networks_dir = Path('/Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/networks')
networks_dir.mkdir(exist_ok=True)

# Determine year range
min_year = int(master_edge_list['first_year'].min())
max_year = int(master_edge_list['first_year'].max())
years = range(max(1958, min_year), min(2025, max_year + 1))

print(f"\nGenerating networks for years {max(1958, min_year)} to {min(2024, max_year)}")
print(f"Total years to process: {len(list(years))}")

# Process each year
for year in tqdm(list(years), desc="Generating yearly networks"):
    
    # Create year directory
    year_dir = networks_dir / str(year)
    year_dir.mkdir(exist_ok=True)
    
    # 1. Yearly Snapshot: Edges where first_year == year
    yearly = master_edge_list[master_edge_list['first_year'] == year].copy()
    yearly_dedup = yearly[['artist_a', 'artist_b']].drop_duplicates()
    yearly_dedup.to_csv(year_dir / 'yearly_snapshot.csv', index=False)
    
    # 2. Cumulative Network: Edges where first_year <= year
    cumulative = master_edge_list[master_edge_list['first_year'] <= year].copy()
    cumulative_dedup = cumulative[['artist_a', 'artist_b']].drop_duplicates()
    cumulative_dedup.to_csv(year_dir / 'cumulative_network.csv', index=False)
    
    # 3. Rolling 10-Year Network: Edges where (year - 9) <= first_year <= year
    rolling_start = year - 9
    rolling = master_edge_list[
        (master_edge_list['first_year'] >= rolling_start) & 
        (master_edge_list['first_year'] <= year)
    ].copy()
    rolling_dedup = rolling[['artist_a', 'artist_b']].drop_duplicates()
    rolling_dedup.to_csv(year_dir / 'rolling_10year.csv', index=False)

print("\n" + "="*80)
print("PROCESSING COMPLETE")
print("="*80)

# Summary statistics
print("\nSummary Statistics:")
print(f"  Master edge list: {len(master_edge_list):,} total collaboration records")
print(f"  Unique artist pairs: {master_edge_list[['artist_a', 'artist_b']].drop_duplicates().shape[0]:,}")
print(f"  Year range: {min_year} - {max_year}")
print(f"  Networks generated: {len(list(years))} years × 3 network types = {len(list(years)) * 3} files")
print(f"\nOutput locations:")
print(f"  Master edge list: {output_path}")
print(f"  Yearly networks: {networks_dir}/[YEAR]/")
print(f"    - yearly_snapshot.csv (edges from that year only)")
print(f"    - cumulative_network.csv (all edges up to that year)")
print(f"    - rolling_10year.csv (edges from previous 10 years)")


Loading Billboard artist IDs...
Total artists in df_artists_top_10_songs_only: 2420
Artists with MusicBrainz IDs: 2041
Artists without MusicBrainz IDs: 379

PHASE 1: Extracting Master Edge List from MusicBrainz

Executing SQL query (this may take several minutes)...


Extracting batches:  20%|██        | 1/5 [00:19<01:16, 19.21s/it]

  Batch 1/5: Found 6,175 edges


Extracting batches:  40%|████      | 2/5 [00:35<00:53, 17.68s/it]

  Batch 2/5: Found 4,569 edges


...

Extracting batches: 100%|██████████| 5/5 [01:21<00:00, 16.28s/it]

  Batch 5/5: Found 577 edges

Total edges extracted: 25,248
Year range: 1939 - 2026
Unique artist pairs: 15,404



Master edge list saved to: /Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/master_edge_list.parquet
Also saved as CSV: /Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/master_edge_list.csv

PHASE 2: Generating Temporal Network Snapshots

Generating networks for years 1958 to 2024
Total years to process: 67


Generating yearly networks: 100%|██████████| 67/67 [00:00<00:00, 239.69it/s]


PROCESSING COMPLETE

Summary Statistics:
  Master edge list: 25,248 total collaboration records
  Unique artist pairs: 15,404
  Year range: 1939 - 2026
  Networks generated: 67 years × 3 network types = 201 files

Output locations:
  Master edge list: /Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/master_edge_list.parquet
  Yearly networks: /Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/networks/[YEAR]/
    - yearly_snapshot.csv (edges from that year only)
    - cumulative_network.csv (all edges up to that year)
    - rolling_10year.csv (edges from previous 10 years)


In [26]:
import pandas as pd
import psycopg2
from psycopg2 import sql
from tqdm import tqdm
import os
from pathlib import Path

# Database connection parameters
DB_PARAMS = {
    'dbname': 'musicbrainz_db',
    'user': 'musicbrainz',
    'password': 'musicbrainz',
    'host': 'localhost',
    'port': 5432
}

# ============================================================================
# LOAD BILLBOARD ARTIST IDs (ALL ARTISTS)
# ============================================================================

print("Loading Billboard artist IDs (ALL ARTISTS)...")
df_artists = pd.read_csv('/Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/df_artists.csv')

# Get artist IDs and filter out NaN values, convert to integers
artist_ids = df_artists['musicbrainz_artist_id'].dropna().astype(int).unique().tolist()

print(f"Total artists in df_artists: {len(df_artists)}")
print(f"Artists with MusicBrainz IDs: {len(artist_ids)}")
print(f"Artists without MusicBrainz IDs: {df_artists['musicbrainz_artist_id'].isna().sum()}")

# ============================================================================
# PHASE 1: Extract Master Edge List
# ============================================================================

print("\n" + "="*80)
print("PHASE 1: Extracting Master Edge List from MusicBrainz")
print("="*80)

def extract_master_edge_list(artist_ids, batch_size=500):
    """
    Extract all collaboration edges for the given artist IDs.
    Uses batching to avoid memory issues with large IN clauses.
    """
    
    # SQL query template - Uses INTEGER type, not UUID
    sql_query = """
    WITH EarliestRG AS (
        -- Anchor every album/single to its original release year
        SELECT r.release_group, MIN(rc.date_year) as first_year
        FROM release r
        JOIN release_country rc ON r.id = rc.release
        WHERE rc.date_year IS NOT NULL
        GROUP BY r.release_group
    ),
    CollaborationEdges AS (
        -- Find every unique pair of artists on the same credit
        SELECT 
            acn1.artist AS artist_a,
            acn2.artist AS artist_b,
            rg.id AS release_group_id,
            erg.first_year
        FROM artist_credit_name acn1
        JOIN artist_credit_name acn2 ON acn1.artist_credit = acn2.artist_credit
        JOIN release_group rg ON rg.artist_credit = acn1.artist_credit
        JOIN EarliestRG erg ON rg.id = erg.release_group
        WHERE acn1.artist = ANY(%s::integer[])
          AND acn1.artist < acn2.artist
    )
    SELECT * FROM CollaborationEdges;
    """
    
    all_edges = []
    
    # Process in batches
    num_batches = (len(artist_ids) + batch_size - 1) // batch_size
    
    conn = psycopg2.connect(**DB_PARAMS)
    conn.autocommit = True
    
    with conn.cursor() as cur:
        for i in tqdm(range(num_batches), desc="Extracting batches"):
            batch_ids = artist_ids[i*batch_size:(i+1)*batch_size]
            
            # Execute query for this batch
            cur.execute(sql_query, (batch_ids,))
            
            # Fetch results
            batch_results = cur.fetchall()
            all_edges.extend(batch_results)
            
            print(f"  Batch {i+1}/{num_batches}: Found {len(batch_results):,} edges")
    
    conn.close()
    
    # Convert to DataFrame
    df_edges = pd.DataFrame(all_edges, columns=['artist_a', 'artist_b', 'release_group_id', 'first_year'])
    
    return df_edges

# Extract the master edge list
print("\nExecuting SQL query (this may take several minutes)...")
master_edge_list = extract_master_edge_list(artist_ids)

print(f"\nTotal edges extracted: {len(master_edge_list):,}")
print(f"Year range: {master_edge_list['first_year'].min()} - {master_edge_list['first_year'].max()}")
print(f"Unique artist pairs: {master_edge_list[['artist_a', 'artist_b']].drop_duplicates().shape[0]:,}")

# Save to parquet
output_path = '/Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/master_edge_list_all_artists.parquet'
master_edge_list.to_parquet(output_path, index=False)
print(f"\nMaster edge list saved to: {output_path}")

# Also save as CSV for easy inspection
csv_path = '/Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/master_edge_list_all_artists.csv'
master_edge_list.to_csv(csv_path, index=False)
print(f"Also saved as CSV: {csv_path}")

# ============================================================================
# PHASE 2: Generate Temporal Network Snapshots
# ============================================================================

print("\n" + "="*80)
print("PHASE 2: Generating Temporal Network Snapshots")
print("="*80)

# Create output directory structure
networks_dir = Path('/Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/networks_all_artists')
networks_dir.mkdir(exist_ok=True)

# Determine year range
min_year = int(master_edge_list['first_year'].min())
max_year = int(master_edge_list['first_year'].max())
years = range(max(1958, min_year), min(2025, max_year + 1))

print(f"\nGenerating networks for years {max(1958, min_year)} to {min(2024, max_year)}")
print(f"Total years to process: {len(list(years))}")

# Process each year
for year in tqdm(list(years), desc="Generating yearly networks"):
    
    # Create year directory
    year_dir = networks_dir / str(year)
    year_dir.mkdir(exist_ok=True)
    
    # 1. Yearly Snapshot: Edges where first_year == year
    yearly = master_edge_list[master_edge_list['first_year'] == year].copy()
    yearly_dedup = yearly[['artist_a', 'artist_b']].drop_duplicates()
    yearly_dedup.to_csv(year_dir / 'yearly_snapshot.csv', index=False)
    
    # 2. Cumulative Network: Edges where first_year <= year
    cumulative = master_edge_list[master_edge_list['first_year'] <= year].copy()
    cumulative_dedup = cumulative[['artist_a', 'artist_b']].drop_duplicates()
    cumulative_dedup.to_csv(year_dir / 'cumulative_network.csv', index=False)
    
    # 3. Rolling 10-Year Network: Edges where (year - 9) <= first_year <= year
    rolling_start = year - 9
    rolling = master_edge_list[
        (master_edge_list['first_year'] >= rolling_start) & 
        (master_edge_list['first_year'] <= year)
    ].copy()
    rolling_dedup = rolling[['artist_a', 'artist_b']].drop_duplicates()
    rolling_dedup.to_csv(year_dir / 'rolling_10year.csv', index=False)

print("\n" + "="*80)
print("PROCESSING COMPLETE")
print("="*80)

# Summary statistics
print("\nSummary Statistics:")
print(f"  Master edge list: {len(master_edge_list):,} total collaboration records")
print(f"  Unique artist pairs: {master_edge_list[['artist_a', 'artist_b']].drop_duplicates().shape[0]:,}")
print(f"  Year range: {min_year} - {max_year}")
print(f"  Networks generated: {len(list(years))} years × 3 network types = {len(list(years)) * 3} files")
print(f"\nOutput locations:")
print(f"  Master edge list: {output_path}")
print(f"  Yearly networks: {networks_dir}/[YEAR]/")
print(f"    - yearly_snapshot.csv (edges from that year only)")
print(f"    - cumulative_network.csv (all edges up to that year)")
print(f"    - rolling_10year.csv (edges from previous 10 years)")


Loading Billboard artist IDs (ALL ARTISTS)...
Total artists in df_artists: 14226
Artists with MusicBrainz IDs: 11564
Artists without MusicBrainz IDs: 2662

PHASE 1: Extracting Master Edge List from MusicBrainz

Executing SQL query (this may take several minutes)...


Extracting batches:   4%|▍         | 1/24 [00:15<06:06, 15.95s/it]

  Batch 1/24: Found 2,518 edges


Extracting batches:   8%|▊         | 2/24 [00:29<05:19, 14.52s/it]

  Batch 2/24: Found 2,467 edges


...

  Batch 23/24: Found 2,648 edges


Extracting batches: 100%|██████████| 24/24 [05:42<00:00, 14.29s/it]


  Batch 24/24: Found 1,104 edges

Total edges extracted: 65,351
Year range: 1926 - 2026
Unique artist pairs: 42,887

Master edge list saved to: /Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/master_edge_list_all_artists.parquet
Also saved as CSV: /Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/master_edge_list_all_artists.csv

PHASE 2: Generating Temporal Network Snapshots

Generating networks for years 1958 to 2024
Total years to process: 67


Generating yearly networks: 100%|██████████| 67/67 [00:00<00:00, 117.34it/s]


PROCESSING COMPLETE

Summary Statistics:
  Master edge list: 65,351 total collaboration records
  Unique artist pairs: 42,887
  Year range: 1926 - 2026
  Networks generated: 67 years × 3 network types = 201 files

Output locations:
  Master edge list: /Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/master_edge_list_all_artists.parquet
  Yearly networks: /Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/networks_all_artists/[YEAR]/
    - yearly_snapshot.csv (edges from that year only)
    - cumulative_network.csv (all edges up to that year)
    - rolling_10year.csv (edges from previous 10 years)


In [27]:
import pandas as pd
import networkx as nx
from pathlib import Path
from tqdm import tqdm
import warnings

warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION
# ============================================================================
NETWORKS_DIR = Path('/Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/networks_all_artists')
DF_ARTISTS_PATH = '/Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/df_artists.csv'
OUTPUT_PATH = '/Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/df_artists_with_network_metrics.csv'

# ============================================================================
# DATA PREPARATION
# ============================================================================

print("Loading df_artists...")
df_artists = pd.read_csv(DF_ARTISTS_PATH)

# Keep IDs as integers (matching the network edge lists)
df_artists['musicbrainz_artist_id'] = df_artists['musicbrainz_artist_id'].fillna(-1).astype(int)

# Initialize all metric columns with None
metric_names = ['degree', 'closeness', 'harmonic_closeness', 'betweenness', 'eigenvector']
network_types_short = ['yearly', 'rolling10', 'cumulative']

for metric in metric_names:
    for net_type in network_types_short:
        df_artists[f'{metric}_centrality_top10_{net_type}'] = None
        df_artists[f'{metric}_centrality_firstsong_{net_type}'] = None

# Map years to lists of artist indices
print("Building year-to-artist index maps...")
top10_year_map = {}
for idx, row in df_artists.iterrows():
    year = row['first_year_top_10_songs']
    if pd.notna(year):
        year = int(year)
        if year not in top10_year_map:
            top10_year_map[year] = []
        top10_year_map[year].append(idx)

firstsong_year_map = {}
for idx, row in df_artists.iterrows():
    year = row['first_song_year']
    if pd.notna(year):
        year = int(year)
        if year not in firstsong_year_map:
            firstsong_year_map[year] = []
        firstsong_year_map[year].append(idx)

# Get unique sorted years to process
all_relevant_years = sorted(set(top10_year_map.keys()) | set(firstsong_year_map.keys()))
print(f"Found {len(all_relevant_years)} unique years to process")

# ============================================================================
# CENTRALITY COMPUTATION ENGINE
# ============================================================================

def get_all_metrics(G):
    """Calculates all 5 centralities for ALL nodes in G at once."""
    if G is None or len(G) == 0:
        return {m: {} for m in metric_names}
    
    print(f" (Nodes: {len(G):,}, Edges: {len(G.edges()):,})", end="")
    
    metrics = {}
    
    try:
        metrics['degree'] = nx.degree_centrality(G)
    except:
        metrics['degree'] = {}
    
    try:
        metrics['closeness'] = nx.closeness_centrality(G)
    except:
        metrics['closeness'] = {}
    
    try:
        metrics['harmonic_closeness'] = nx.harmonic_centrality(G)
    except:
        metrics['harmonic_closeness'] = {}
    
    try:
        # Sample for large graphs to speed up betweenness
        k = min(len(G), 500) if len(G) > 500 else None
        metrics['betweenness'] = nx.betweenness_centrality(G, k=k)
    except:
        metrics['betweenness'] = {}
    
    try:
        metrics['eigenvector'] = nx.eigenvector_centrality(G, max_iter=1000)
    except:
        metrics['eigenvector'] = {}
        
    return metrics

# ============================================================================
# MAIN LOOP (By Year)
# ============================================================================

network_types = {
    'yearly': 'yearly_snapshot',
    'rolling10': 'rolling_10year',
    'cumulative': 'cumulative_network'
}

for year in tqdm(all_relevant_years, desc="Processing years"):
    print(f"\n{'='*80}\nYear {year}")
    
    for net_key, net_filename in network_types.items():
        
        # 1. Load the network file once
        file_path = NETWORKS_DIR / str(year) / f'{net_filename}.csv'
        if not file_path.exists():
            print(f"  {net_key}: File not found, skipping")
            continue
        
        try:
            edges = pd.read_csv(file_path)
            if len(edges) == 0:
                print(f"  {net_key}: Empty network, skipping")
                continue
                
            G = nx.from_pandas_edgelist(edges, 'artist_a', 'artist_b')
            
            # 2. Compute metrics for every node in this year's graph
            print(f"  {net_key}: Computing metrics...", end="")
            all_node_metrics = get_all_metrics(G)
            print(" ✓")
            
            # 3. Update all artists who had their first Top 10 hit in this year
            if year in top10_year_map:
                for idx in top10_year_map[year]:
                    m_id = int(df_artists.at[idx, 'musicbrainz_artist_id'])
                    if m_id == -1:  # Skip artists without MusicBrainz ID
                        continue
                    
                    for m_name in metric_names:
                        val = all_node_metrics[m_name].get(m_id)
                        df_artists.at[idx, f'{m_name}_centrality_top10_{net_key}'] = val

            # 4. Update all artists who had their first charting song in this year
            if year in firstsong_year_map:
                for idx in firstsong_year_map[year]:
                    m_id = int(df_artists.at[idx, 'musicbrainz_artist_id'])
                    if m_id == -1:  # Skip artists without MusicBrainz ID
                        continue
                    
                    for m_name in metric_names:
                        val = all_node_metrics[m_name].get(m_id)
                        df_artists.at[idx, f'{m_name}_centrality_firstsong_{net_key}'] = val
        
        except Exception as e:
            print(f"  {net_key}: Error - {e}")
            continue

# ============================================================================
# SAVE RESULTS
# ============================================================================

print("\n" + "="*80)
print(f"Saving results to {OUTPUT_PATH}...")
df_artists.to_csv(OUTPUT_PATH, index=False)
print("✓ Complete!")

# Summary
print("\n" + "="*80)
print("SUMMARY")
print("="*80)
print(f"\nMetrics calculated for {len(all_relevant_years)} unique years")
print(f"Total columns in output: {len(df_artists.columns)}")
print(f"\nOutput saved to: {OUTPUT_PATH}")


Loading df_artists...
Building year-to-artist index maps...
Found 69 unique years to process


Processing years:   0%|          | 0/69 [00:00<?, ?it/s]


Year 1958
  yearly: Computing metrics... (Nodes: 197, Edges: 152) ✓
  rolling10: Computing metrics... (Nodes: 691, Edges: 737) ✓
  cumulative: Computing metrics... (Nodes: 873, Edges: 1,034)

Processing years:   1%|▏         | 1/69 [00:00<00:42,  1.62it/s]

 ✓

Year 1959
  yearly: Computing metrics... (Nodes: 181, Edges: 135) ✓
  rolling10: Computing metrics... (Nodes: 759, Edges: 813) ✓
  cumulative: Computing metrics... (Nodes: 960, Edges: 1,136)

...

 ✓

Year 2023
  yearly: Computing metrics... (Nodes: 2,492, Edges: 2,287) ✓
  rolling10: Computing metrics... (Nodes: 12,841, Edges: 20,531) ✓
  cumulative: Computing metrics... (Nodes: 23,929, Edges: 39,467)

Processing years:  96%|█████████▌| 66/69 [1:38:28<24:50, 496.79s/it]

 ✓

Year 2024
  yearly: Computing metrics... (Nodes: 2,393, Edges: 2,113) ✓
  rolling10: Computing metrics... (Nodes: 13,186, Edges: 21,200) ✓
  cumulative: Computing metrics... (Nodes: 24,858, Edges: 41,190)

Processing years: 100%|██████████| 69/69 [1:45:34<00:00, 91.81s/it] 

 ✓

Year 2025
  yearly: File not found, skipping
  rolling10: File not found, skipping
  cumulative: File not found, skipping

Year 2026
  yearly: File not found, skipping
  rolling10: File not found, skipping
  cumulative: File not found, skipping

Saving results to /Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/df_artists_with_network_metrics.csv...
✓ Complete!

SUMMARY

Metrics calculated for 69 unique years
Total columns in output: 85

Output saved to: /Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/df_artists_with_network_metrics.csv


In [28]:
import pandas as pd

# ============================================================================
# LOAD DATA
# ============================================================================

print("Loading dataframes...")
df_artists_with_metrics = pd.read_csv('/Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/df_artists_with_network_metrics.csv')
df_artists_top_10 = pd.read_csv('/Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/df_artists_top_10_songs_only.csv')

print(f"df_artists_with_metrics: {df_artists_with_metrics.shape}")
print(f"df_artists_top_10_songs_only: {df_artists_top_10.shape}")

# ============================================================================
# IDENTIFY NETWORK METRIC COLUMNS
# ============================================================================

# Get all the network metric column names
network_columns = [col for col in df_artists_with_metrics.columns 
                   if '_centrality_' in col]

print(f"\nFound {len(network_columns)} network metric columns to transfer")

# ============================================================================
# MERGE DATAFRAMES
# ============================================================================

print("\nMerging dataframes on 'performer_normalized'...")

# Select only the columns we need from df_artists_with_metrics
columns_to_merge = ['performer_normalized'] + network_columns
df_metrics_subset = df_artists_with_metrics[columns_to_merge]

# Merge with df_artists_top_10
df_result = df_artists_top_10.merge(
    df_metrics_subset,
    on='performer_normalized',
    how='left'
)

print(f"Result shape: {df_result.shape}")

# ============================================================================
# VERIFY MERGE
# ============================================================================

# Check how many artists got matched
matched = df_result[network_columns[0]].notna().sum()
total = len(df_result)
print(f"\nMetrics added for {matched}/{total} artists ({matched/total*100:.1f}%)")

# ============================================================================
# SAVE RESULTS
# ============================================================================

output_path = '/Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/df_artists_top_10_songs_only_with_network_metrics.csv'
print(f"\nSaving to {output_path}...")
df_result.to_csv(output_path, index=False)

print("✓ Complete!")

# ============================================================================
# SUMMARY
# ============================================================================

print("\n" + "="*80)
print("SUMMARY")
print("="*80)
print(f"Original df_artists_top_10_songs_only columns: {len(df_artists_top_10.columns)}")
print(f"Network metric columns added: {len(network_columns)}")
print(f"New total columns: {len(df_result.columns)}")
print(f"\nOutput file: df_artists_top_10_songs_only_with_network_metrics.csv")


Loading dataframes...
df_artists_with_metrics: (14226, 85)
df_artists_top_10_songs_only: (2420, 55)

Found 30 network metric columns to transfer

Merging dataframes on 'performer_normalized'...
Result shape: (2420, 85)

Metrics added for 407/2420 artists (16.8%)

Saving to /Users/jamesemcnally/Documents/GitHub/spring-2026-hitmakers/df_artists_top_10_songs_only_with_network_metrics.csv...
✓ Complete!

SUMMARY
Original df_artists_top_10_songs_only columns: 55
Network metric columns added: 30
New total columns: 85

Output file: df_artists_top_10_songs_only_with_network_metrics.csv
